<h1>Databases

In [7]:
install.packages("nycflights13", repos="http://cran.us.r-project.org/")
library("nycflights13")
library(dplyr)


Create a database<br\>

In [2]:
my_db <- src_sqlite("my_db.sqlite3", create = T)


src : It is a collection of types of database tables. Use src_sqlite(), src_mysql(), src_postgres() and src_bigquery() to connect to the specific types supported by dplyr

Import flights data to the database my_db

In [3]:
library(nycflights13)
head(flights)
flights_sqlite <- copy_to(my_db, flights, temporary = FALSE, indexes = list( c("year", "month", "day"), "carrier", "tailnum"))
#returns a table object in remote source
#copy_to function copies the flights data from the flight dataframe to my_db database
#indexing is done for faster retrieval
# c("year", "month", "day") are saves as multitable index
#while "carrier", "tailnum" are stored as seperate index
#more on indexing : http://www.sqlite.org/queryplanner.html


year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
2013,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,2013-01-01 05:00:00
2013,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,2013-01-01 05:00:00
2013,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,2013-01-01 05:00:00
2013,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45,2013-01-01 05:00:00
2013,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,2013-01-01 06:00:00
2013,1,1,554,558,-4,740,728,12,UA,1696,N39463,EWR,ORD,150,719,5,58,2013-01-01 05:00:00


In [4]:
#copy_to(dest, df, name = deparse(substitute(df)), ...)
#name name for new remote table

In [5]:
class(flights_sqlite)
head(flights_sqlite)

[1] "tbl_sqlite" "tbl_sql"    "tbl_lazy"   "tbl"

Source:   query [?? x 19]
Database: sqlite 3.11.1 [my_db.sqlite3]

   year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
  <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
1  2013     1     1      517            515         2      830            819
2  2013     1     1      533            529         4      850            830
3  2013     1     1      542            540         2      923            850
4  2013     1     1      544            545        -1     1004           1022
5  2013     1     1      554            600        -6      812            837
6  2013     1     1      554            558        -4      740            728
# ... with 11 more variables: arr_delay <dbl>, carrier <chr>, flight <int>,
#   tailnum <chr>, origin <chr>, dest <chr>, air_time <dbl>, distance <dbl>,
#   hour <dbl>, minute <dbl>, time_hour <dbl>

Select all the columns from year to day , dep_delay and arr_delay from flights table

In [6]:
select(flights_sqlite, year:day, dep_delay, arr_delay)

Source:   query [?? x 5]
Database: sqlite 3.11.1 [my_db.sqlite3]

    year month   day dep_delay arr_delay
   <int> <int> <int>     <dbl>     <dbl>
1   2013     1     1         2        11
2   2013     1     1         4        20
3   2013     1     1         2        33
4   2013     1     1        -1       -18
5   2013     1     1        -6       -25
6   2013     1     1        -4        12
7   2013     1     1        -5        19
8   2013     1     1        -3       -14
9   2013     1     1        -3        -8
10  2013     1     1        -2         8
# ... with more rows

Filter the rows with dep_delay<240filter(flights_sqlite, dep_delay > 240)filter(flights_sqlite, dep_delay > 240)

In [8]:
filter(flights_sqlite, dep_delay > 240)

Source:   query [?? x 19]
Database: sqlite 3.11.1 [my_db.sqlite3]

    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
1   2013     1     1      848           1835       853     1001           1950
2   2013     1     1     1815           1325       290     2120           1542
3   2013     1     1     1842           1422       260     1958           1535
4   2013     1     1     2115           1700       255     2330           1920
5   2013     1     1     2205           1720       285       46           2040
6   2013     1     1     2343           1724       379      314           1938
7   2013     1     2     1332            904       268     1616           1128
8   2013     1     2     1412            838       334     1710           1147
9   2013     1     2     1607           1030       337     2003           1355
10  2013     1     2     2131           1512       379     2340 

Arrange or sort in ascending  or descending order

In [24]:
head(arrange(flights_sqlite, desc(day)))
head(arrange(flights_sqlite, year, month, day))


Source:   query [?? x 19]
Database: sqlite 3.11.1 [my_db.sqlite3]

   year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
  <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
1  2013     1    31        1           2100       181      124           2225
2  2013     1    31        4           2359         5      455            444
3  2013     1    31        7           2359         8      453            437
4  2013     1    31       12           2250        82      132              7
5  2013     1    31       26           2154       152      328             50
6  2013     1    31       34           2159       155      135           2315
# ... with 11 more variables: arr_delay <dbl>, carrier <chr>, flight <int>,
#   tailnum <chr>, origin <chr>, dest <chr>, air_time <dbl>, distance <dbl>,
#   hour <dbl>, minute <dbl>, time_hour <dbl>

Source:   query [?? x 19]
Database: sqlite 3.11.1 [my_db.sqlite3]

   year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
  <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
1  2013     1     1      517            515         2      830            819
2  2013     1     1      533            529         4      850            830
3  2013     1     1      542            540         2      923            850
4  2013     1     1      544            545        -1     1004           1022
5  2013     1     1      554            600        -6      812            837
6  2013     1     1      554            558        -4      740            728
# ... with 11 more variables: arr_delay <dbl>, carrier <chr>, flight <int>,
#   tailnum <chr>, origin <chr>, dest <chr>, air_time <dbl>, distance <dbl>,
#   hour <dbl>, minute <dbl>, time_hour <dbl>

Get a new computed column

In [54]:
# TimeofFlight=transmute(flights_sqlite, TimeofFlight = arr_time - dep_time)
#class(TimeofFlight)
flightstime=mutate(flights_sqlite,TimeofFlight = arr_time - dep_time)%>%
select(TimeofFlight )
head(flightstime)

Source:   query [?? x 1]
Database: sqlite 3.11.1 [my_db.sqlite3]

  TimeofFlight
         <int>
1          313
2          317
3          381
4          460
5          258
6          186

In [45]:
class(flights_sqlite)

[1] "tbl_sqlite" "tbl_sql"    "tbl_lazy"   "tbl"

Summarise all rows into one column based on the the mean of the time of flights

In [55]:
summarise(flightstime, delay = mean(TimeofFlight))

Source:   query [?? x 1]
Database: sqlite 3.11.1 [my_db.sqlite3]

     delay
     <dbl>
1 153.1973

# Laziness

In [56]:
c1 <- filter(flights_sqlite, year == 2013, month == 1, day == 1)
c2 <- select(c1, year, month, day, carrier, dep_delay, air_time, distance)
c3 <- mutate(c2, speed = distance / air_time * 60)
c4 <- arrange(c3, year, month, day, carrier)

The above sequence dosent actually fire anything to database unless we specifically specify.


To pull down all the results use collect():

In [64]:
collect(flights_sqlite,n=2)

Warning message:
"Only first 2 results retrieved. Use n = Inf to retrieve all."

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
2013,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,1357016400
2013,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,1357016400


Explaind the command dplyr actually fires

In [65]:
explain(select(flights_sqlite,year=12))

<SQL>
SELECT `tailnum` AS `year`
FROM `flights`


<PLAN>
   addr      opcode p1    p2 p3      p4 p5 comment
1     0        Init  0     8  0         00      NA
2     1    OpenRead  1 31146  0  k(2,,) 00      NA
3     2      Rewind  1     6  1       0 00      NA
4     3      Column  1     0  1         00      NA
5     4   ResultRow  1     1  0         00      NA
6     5        Next  1     3  0         01      NA
7     6       Close  1     0  0         00      NA
8     7        Halt  0     0  0         00      NA
9     8 Transaction  0     0  5       0 01      NA
10    9   TableLock  0     2  0 flights 00      NA
11   10        Goto  0     1  0         00      NA


Some useful commands<br\>There are three ways to force the computation of a query:<br\>
collect() executes the query and returns the results to R.<br\>
compute() executes the query and stores the results in a temporary table in the database.<br\>
collapse() turns the query into a table expression.

To return the last few rows : tail()

In [75]:
tail(collect(flightstime,n=10)) #returns last five rows of first five rows

Warning message:
"Only first 10 results retrieved. Use n = Inf to retrieve all."

TimeofFlight
258
186
358
152
281
195


To translate a mathematical function to sql format for querying

In [76]:

translate_sql(x == 1 && (y < 2 || z > 3))

<SQL> "x" = 1.0 AND ("y" < 2.0 OR "z" > 3.0)

# Grouping on tables

In [77]:
by_tailnum <- group_by(flights_sqlite, tailnum)
delay <- summarise(by_tailnum,
  count = n(),
  dist = mean(distance),
  delay = mean(arr_delay)
)
delay <- filter(delay, count > 20, dist < 2000)
delay_local <- collect(delay)

In [78]:
delay

Source:   query [?? x 4]
Database: sqlite 3.11.1 [my_db.sqlite3]

   tailnum count     dist      delay
     <chr> <int>    <dbl>      <dbl>
1     <NA>  2512 710.2576         NA
2   N0EGMQ   371 676.1887  9.9829545
3   N10156   153 757.9477 12.7172414
4   N102UW    48 535.8750  2.9375000
5   N103US    46 535.1957 -6.9347826
6   N104UW    47 535.2553  1.8043478
7   N10575   289 519.7024 20.6914498
8   N105UW    45 524.8444 -0.2666667
9   N107US    41 528.7073 -5.7317073
10  N108UW    60 534.5000 -1.2500000
# ... with more rows